In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# reading the file

file_path1 = r"C:\Users\HP\Downloads\dataset-1.csv"
file_path2 = r"C:\Users\HP\Downloads\dataset-2.csv"
file_path3 = r"C:\Users\HP\Downloads\dataset-3.csv"

dataset_1 = pd.read_csv(file_path1)
dataset_2 = pd.read_csv(file_path2)
dataset_3 = pd.read_csv(file_path3)

Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [6]:
import numpy as np
from scipy.sparse.csgraph import floyd_warshall

# Extract unique IDs
unique_ids = np.unique(dataset_3[['id_start', 'id_end']].values)

# Initialize a matrix with inf for all entries
size = len(unique_ids)
distance_matrix = np.full((size, size), np.inf)

# Populate the matrix with the given distances
for _, row in dataset_3.iterrows():
    start_index = np.where(unique_ids == row['id_start'])[0][0]
    end_index = np.where(unique_ids == row['id_end'])[0][0]
    distance_matrix[start_index, end_index] = row['distance']
    distance_matrix[end_index, start_index] = row['distance']  # Ensure symmetry

# Set diagonal to 0
np.fill_diagonal(distance_matrix, 0)

# Calculate cumulative distances for indirect routes using Floyd-Warshall algorithm
cumulative_distance_matrix = floyd_warshall(distance_matrix, directed=False)

# Convert the cumulative distance matrix to a DataFrame
result_df = pd.DataFrame(cumulative_distance_matrix, index=unique_ids, columns=unique_ids)

# Display the head of the resulting DataFrame
result_df.head()

,1001400,1001402,1001404,1001406,1001408,1001410,1001412,1001414,1001416,1001418,...,1001462,1001464,1001466,1001468,1001470,1001472,1001488,1004354,1004355,1004356
1001400,0.0,9.7,29.9,45.9,67.6,78.7,94.3,112.5,125.7,139.3,...,371.8,398.5,407.0,417.7,428.3,444.3,264.5,270.5,272.5,268.5
1001402,9.7,0.0,20.2,36.2,57.9,69.0,84.6,102.8,116.0,129.6,...,362.1,388.8,397.3,408.0,418.6,434.6,254.8,260.8,262.8,258.8
1001404,29.9,20.2,0.0,16.0,37.7,48.8,64.4,82.6,95.8,109.4,...,341.9,368.6,377.1,387.8,398.4,414.4,234.6,240.6,242.6,238.6
1001406,45.9,36.2,16.0,0.0,21.7,32.8,48.4,66.6,79.8,93.4,...,325.9,352.6,361.1,371.8,382.4,398.4,218.6,224.6,226.6,222.6
1001408,67.6,57.9,37.7,21.7,0.0,11.1,26.7,44.9,58.1,71.7,...,304.2,330.9,339.4,350.1,360.7,376.7,196.9,202.9,204.9,200.9


Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [7]:
# Unroll the distance matrix
unrolled_distance_matrix = result_df.unstack().reset_index()
unrolled_distance_matrix.columns = ['id_start', 'id_end', 'distance']

# Display the head of the resulting DataFrame
unrolled_distance_matrix.head()

,id_start,id_end,distance
0,1001400,1001400,0.0
1,1001400,1001402,9.7
2,1001400,1001404,29.9
3,1001400,1001406,45.9
4,1001400,1001408,67.6


Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [8]:
# Calculate the average distance for the reference value
reference_value = 1001400
average_distance = unrolled_distance_matrix[unrolled_distance_matrix['id_start'] == reference_value]['distance'].mean()

# Find values within 10% threshold
threshold = 0.1 * average_distance
lower_bound = average_distance - threshold
upper_bound = average_distance + threshold
values_within_threshold = unrolled_distance_matrix[(unrolled_distance_matrix['id_start'] != reference_value) & (unrolled_distance_matrix['distance'] >= lower_bound) & (unrolled_distance_matrix['distance'] <= upper_bound)]['id_start'].unique()

# Sort the list of values
sorted_values = sorted(values_within_threshold)
sorted_values

[1001402,
 1001404,
 1001406,
 1001408,
 1001410,
 1001412,
 1001414,
 1001416,
 1001418,
 1001420,
 1001422,
 1001424,
 1001426,
 1001428,
 1001430,
 1001432,
 1001434,
 1001436,
 1001437,
 1001438,
 1001440,
 1001442,
 1001444,
 1001446,
 1001448,
 1001450,
 1001452,
 1001454,
 1001456,
 1001458,
 1001460,
 1001461,
 1001462,
 1001464,
 1001466,
 1001468,
 1001470,
 1001472,
 1001488,
 1004354,
 1004355,
 1004356]

Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [9]:
# Calculate toll rates based on vehicle types
rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

def calculate_toll_rate(df):
    for vehicle, rate in rate_coefficients.items():
        df[vehicle] = df['distance'] * rate
    return df

# Apply the function to the input DataFrame
result_with_toll_rates = calculate_toll_rate(unrolled_distance_matrix)

# Display the head of the resulting DataFrame
result_with_toll_rates.head()

,id_start,id_end,distance,moto,car,rv,bus,truck
0,1001400,1001400,0.0,0.00,0.00,0.00,0.00,0.00
1,1001400,1001402,9.7,7.76,11.64,14.55,21.34,34.92
2,1001400,1001404,29.9,23.92,35.88,44.85,65.78,107.64
3,1001400,1001406,45.9,36.72,55.08,68.85,100.98,165.24
4,1001400,1001408,67.6,54.08,81.12,101.40,148.72,243.36


Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [10]:
# Calculate time-based toll rates
import datetime

def calculate_time_based_toll_rates(df):
    # Define the time ranges
    weekday_morning = (datetime.time(0, 0, 0), datetime.time(10, 0, 0))
    weekday_afternoon = (datetime.time(10, 0, 0), datetime.time(18, 0, 0))
    weekday_evening = (datetime.time(18, 0, 0), datetime.time(23, 59, 59))
    weekend_all_day = (datetime.time(0, 0, 0), datetime.time(23, 59, 59))
    
    # Apply discount factors based on time ranges
    df['start_day'] = 'Monday'
    df['end_day'] = 'Sunday'
    df['start_time'] = datetime.time(0, 0, 0)
    df['end_time'] = datetime.time(23, 59, 59)
    df['moto'] = df.apply(lambda row: row['moto'] * 0.8 if (row['start_time'] >= weekday_morning[0] and row['end_time'] <= weekday_morning[1]) or (row['start_time'] >= weekday_evening[0] and row['end_time'] <= weekday_evening[1]) else (row['moto'] * 1.2 if row['start_time'] >= weekday_afternoon[0] and row['end_time'] <= weekday_afternoon[1] else row['moto'] * 0.7), axis=1)
    df['car'] = df.apply(lambda row: row['car'] * 0.8 if (row['start_time'] >= weekday_morning[0] and row['end_time'] <= weekday_morning[1]) or (row['start_time'] >= weekday_evening[0] and row['end_time'] <= weekday_evening[1]) else (row['car'] * 1.2 if row['start_time'] >= weekday_afternoon[0] and row['end_time'] <= weekday_afternoon[1] else row['car'] * 0.7), axis=1)
    df['rv'] = df.apply(lambda row: row['rv'] * 0.8 if (row['start_time'] >= weekday_morning[0] and row['end_time'] <= weekday_morning[1]) or (row['start_time'] >= weekday_evening[0] and row['end_time'] <= weekday_evening[1]) else (row['rv'] * 1.2 if row['start_time'] >= weekday_afternoon[0] and row['end_time'] <= weekday_afternoon[1] else row['rv'] * 0.7), axis=1)
    df['bus'] = df.apply(lambda row: row['bus'] * 0.8 if (row['start_time'] >= weekday_morning[0] and row['end_time'] <= weekday_morning[1]) or (row['start_time'] >= weekday_evening[0] and row['end_time'] <= weekday_evening[1]) else (row['bus'] * 1.2 if row['start_time'] >= weekday_afternoon[0] and row['end_time'] <= weekday_afternoon[1] else row['bus'] * 0.7), axis=1)
    df['truck'] = df.apply(lambda row: row['truck'] * 0.8 if (row['start_time'] >= weekday_morning[0] and row['end_time'] <= weekday_morning[1]) or (row['start_time'] >= weekday_evening[0] and row['end_time'] <= weekday_evening[1]) else (row['truck'] * 1.2 if row['start_time'] >= weekday_afternoon[0] and row['end_time'] <= weekday_afternoon[1] else row['truck'] * 0.7), axis=1)
    return df

# Apply the function to the input DataFrame
result_with_time_based_toll_rates = calculate_time_based_toll_rates(result_with_toll_rates)

# Display the head of the resulting DataFrame
result_with_time_based_toll_rates.head()

,id_start,id_end,distance,moto,car,rv,bus,truck,start_day,end_day,start_time,end_time
0,1001400,1001400,0.0,0.000,0.000,0.000,0.000,0.000,Monday,Sunday,00:00:00,23:59:59
1,1001400,1001402,9.7,5.432,8.148,10.185,14.938,24.444,Monday,Sunday,00:00:00,23:59:59
2,1001400,1001404,29.9,16.744,25.116,31.395,46.046,75.348,Monday,Sunday,00:00:00,23:59:59
3,1001400,1001406,45.9,25.704,38.556,48.195,70.686,115.668,Monday,Sunday,00:00:00,23:59:59
4,1001400,1001408,67.6,37.856,56.784,70.980,104.104,170.352,Monday,Sunday,00:00:00,23:59:59
